<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Potentially-Fraudulent-Providers" data-toc-modified-id="Potentially-Fraudulent-Providers-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Potentially Fraudulent Providers</a></span></li><li><span><a href="#Not-Fraudulent-Providers" data-toc-modified-id="Not-Fraudulent-Providers-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Not Fraudulent Providers</a></span></li></ul></div>

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.insert(0, '..')
from joblib import load
import Functions as fxns
from hashable_df import hashable_df
%matplotlib inline
plt.rcParams['figure.figsize'] = (9, 6)
sns.set(style = "whitegrid")
sns.set_palette("icefire")
pd.set_option('display.max_columns', 500)
import warnings
warnings.filterwarnings(action="ignore")

In [4]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [5]:
claims = load('../claims.pkl')

In [7]:
# Create variables for convenience 
diag_code = claims.columns[claims.columns.str.contains('DiagnosisCode')].tolist()
proc_code = claims.columns[claims.columns.str.contains('ProcedureCode')].tolist()
codes = diag_code + proc_code
chronic = claims.columns[claims.columns.str.contains("Chronic")].tolist()

# Potentially Fraudulent Providers

In [8]:
frequent_chronic_fraud = apriori(claims[claims['PotentialFraud'] == 1][chronic], 
                                 min_support = 0.1, use_colnames = True)
rules_fraud = association_rules(frequent_chronic_fraud)

In [9]:
rules_fraud.sort_values(by = 'confidence', ascending = False).head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
204,"(ChronicCond_KidneyDisease, ChronicCond_ObstrP...",(ChronicCond_IschemicHeart),0.155703,0.763699,0.147658,0.948329,1.241759,0.028748,4.573233
197,"(ChronicCond_KidneyDisease, ChronicCond_Alzhei...",(ChronicCond_IschemicHeart),0.106614,0.763699,0.100843,0.945872,1.238541,0.019422,4.365611
227,"(ChronicCond_KidneyDisease, ChronicCond_Ischem...",(ChronicCond_Diabetes),0.109170,0.708495,0.103099,0.944385,1.332944,0.025752,5.241441
190,"(ChronicCond_ObstrPulmonary, ChronicCond_Alzhe...",(ChronicCond_IschemicHeart),0.128010,0.763699,0.120576,0.941924,1.233371,0.022815,4.068806
181,"(ChronicCond_KidneyDisease, ChronicCond_Alzhei...",(ChronicCond_Diabetes),0.109983,0.708495,0.103512,0.941164,1.328398,0.025590,4.954519


In [10]:
provider_code = claims.groupby(['Provider','ClmAdmitDiagnosisCode']).size().unstack().reset_index().fillna(0).set_index('Provider')

In [11]:
provider_code = provider_code.apply(lambda x: np.where(x >= 1, 1, 0))

In [12]:
provider_freq_code = apriori(provider_code, 
                                 min_support = 0.05, use_colnames = True)
provider_freq_code = association_rules(provider_freq_code)

In [13]:
provider_freq_code

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(4019, 1629)",(42731),0.060508,0.313836,0.050827,0.840000,2.676555,0.031837,4.288524
1,"(4019, 1629)",(V7612),0.060508,0.324526,0.051432,0.850000,2.619204,0.031796,4.503160
2,"(42731, 1629)",(V7612),0.066559,0.324526,0.054457,0.818182,2.521159,0.032857,3.715107
3,"(1629, V7612)",(42731),0.065147,0.313836,0.054457,0.835913,2.663534,0.034012,4.181715
4,"(2449, 25000)",(V7612),0.058895,0.324526,0.051029,0.866438,2.669858,0.031916,5.057395
...,...,...,...,...,...,...,...,...,...
2923,"(V5883, 78605, V7612, 78900, 42731)",(4019),0.056878,0.260387,0.051432,0.904255,3.472733,0.036622,7.724844
2924,"(V5883, 4019, V7612, 78900, 42731)",(78605),0.061517,0.285397,0.051432,0.836066,2.929479,0.033875,4.359076
2925,"(78605, 4019, V5883, 78900)","(42731, V7612)",0.059096,0.176079,0.051432,0.870307,4.942707,0.041026,6.352864
2926,"(78605, V5883, 78900, V7612)","(42731, 4019)",0.062122,0.151876,0.051432,0.827922,5.451312,0.041997,4.928722


# Not Fraudulent Providers

In [14]:
frequent_chronic_notfraud = apriori(claims[claims['PotentialFraud'] == 0][chronic], 
                                 min_support = 0.1, use_colnames = True)
rules_notfraud = association_rules(frequent_chronic_notfraud)

In [15]:
rules_notfraud.sort_values(by = 'confidence', ascending = False).head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
187,"(ChronicCond_KidneyDisease, ChronicCond_ObstrP...",(ChronicCond_IschemicHeart),0.145434,0.756533,0.137629,0.946332,1.250880,0.027603,4.536551
201,"(ChronicCond_rheumatoidarthritis, ChronicCond_...",(ChronicCond_IschemicHeart),0.116324,0.756533,0.109694,0.943006,1.246484,0.021691,4.271828
110,"(ChronicCond_KidneyDisease, ChronicCond_ObstrP...",(ChronicCond_IschemicHeart),0.156661,0.756533,0.147313,0.940329,1.242944,0.028793,4.080119
176,"(ChronicCond_ObstrPulmonary, ChronicCond_Alzhe...",(ChronicCond_IschemicHeart),0.122554,0.756533,0.114798,0.936715,1.238167,0.022082,3.847117
164,"(ChronicCond_ObstrPulmonary, ChronicCond_rheum...",(ChronicCond_IschemicHeart),0.110363,0.756533,0.103345,0.936413,1.237768,0.019852,3.828873
